<a href="https://colab.research.google.com/github/zaellis/270Computer/blob/master/test_flow4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Digital inverter with OpenLane

```
Copyright 2022 Google LLC.
SPDX-License-Identifier: Apache-2.0
```

Run a simple digital inverter design thru the [OpenLane](https://github.com/The-OpenROAD-Project/OpenLane/) GDS to RTL flow targeting the [open source SKY130 PDK](https://github.com/google/skywater-pdk/).

In [27]:
#@title Install Dependencies {display-mode: "form"}
#@markdown Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

#@markdown Main components we will install

#@markdown *   Open_pdks.sky130a : a PDK installer for open-source EDA tools.
#@markdown *   Openlane : an automated RTL to GDSII flow based on several components including OpenROAD, Yosys, Magic, Netgen, CVC, SPEF-Extractor, KLayout and a number of custom scripts for design exploration and optimization.
#@markdown *   GDSTK : a C++ library for creation and manipulation of GDSII and OASIS files. 

#@title Install dependencies {display-mode: "form"}
#@markdown - Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

openlane_version = 'latest' #@param {type:"string"}
open_pdks_version = 'latest' #@param {type:"string"}

if openlane_version == 'latest':
  openlane_version = ''
if open_pdks_version == 'latest':
  open_pdks_version = ''

import os
import pathlib
import sys

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     openlane={openlane_version} \
                     open_pdks.sky130a={open_pdks_version}
!python -m pip install --force-reinstall --no-binary gdstk gdstk cairosvg
PATH = os.environ['PATH']
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}
site_package_path = conda_prefix_path / 'lib/python3.7/site-packages'
sys.path.append(str(site_package_path.resolve()))


                                           __
          __  ______ ___  ____ _____ ___  / /_  ____ _
         / / / / __ `__ \/ __ `/ __ `__ \/ __ \/ __ `/
        / /_/ / / / / / / /_/ / / / / / / /_/ / /_/ /
       / .___/_/ /_/ /_/\__,_/_/ /_/ /_/_.___/\__,_/
      /_/

Empty environment created at prefix: /content/conda-env

                                           __
          __  ______ ___  ____ _____ ___  / /_  ____ _
         / / / / __ `__ \/ __ `/ __ `__ \/ __ \/ __ `/
        / /_/ / / / / / / /_/ / / / / / / /_/ / /_/ /
       / .___/_/ /_/ /_/\__,_/_/ /_/ /_/_.___/\__,_/
      /_/


Pinned packages:
  - python 3.7*


Transaction

  Prefix: /content/conda-env

  Updating specs:

   - openlane=*
   - open_pdks.sky130a=*


  Package                              Version  Build                 Channel                  Size
─────────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
────────────────────────────────────────

## Retrieve Design Files

In [17]:
!rm -rf ASCON_code-a-chip/
!git clone https://github.com/zaellis/ASCON_code-a-chip.git

Cloning into 'ASCON_code-a-chip'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 37 (delta 8), reused 34 (delta 8), pack-reused 0
Unpacking objects: 100% (37/37), 49.50 KiB | 1.08 MiB/s, done.


## Write configuration

[Documentation](https://openlane.readthedocs.io/en/latest/reference/configuration.html)

In [3]:
%%writefile config.json
{
    "DESIGN_NAME": "ASCON_AEAD",
    "VERILOG_FILES": "/content/ASCON_code-a-chip/RTL/ASCON_AEAD.sv  /content/ASCON_code-a-chip/RTL/ASCON_CONTROLER.sv /content/ASCON_code-a-chip/RTL/ASCON_DATAPATH.sv /content/ASCON_code-a-chip/RTL/ASCON_ROUND_CONSTANT.sv /content/ASCON_code-a-chip/RTL/ASCON_ROUND_FUNCTION.v /content/ASCON_code-a-chip/RTL/ASCON_SBOX.v /content/ASCON_code-a-chip/RTL/ASCONpadding.sv",
    "CLOCK_PERIOD": 20,
    "CLOCK_NET": "clk",
    "CLOCK_PORT": "clk",
    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 1000 1000",
    "PL_TARGET_DENSITY": 0.75
}

Writing config.json


## Run OpenLane Flow

[OpenLane](https://openlane.readthedocs.io/en/latest/) is an automated [RTL](https://en.wikipedia.org/wiki/Register-transfer_level) to [GDSII](https://en.wikipedia.org/wiki/GDSII) flow based on several components including [OpenROAD](https://theopenroadproject.org/), [Yosys](https://yosyshq.net/yosys/), [Magic](http://www.opencircuitdesign.com/magic/), [Netgen](http://opencircuitdesign.com/netgen/) and custom methodology scripts for design exploration and optimization targeting [open source PDKs](https://github.com/google/open-source-pdks).

![img](https://openlane.readthedocs.io/en/latest/_images/flow_v1.png)

In [18]:
%env PDK=sky130A
!flow.tcl -design .

env: PDK=sky130A
OpenLane 2023.03.01_0_ge10820ec-conda
All rights reserved. (c) 2020-2022 Efabless Corporation and contributors.
Available under the Apache License, version 2.0. See the LICENSE file for more details.

[INFO]: Using configuration in 'config.json'...
[INFO]: PDK Root: /content/conda-env/share/pdk
[INFO]: Process Design Kit: sky130A
[INFO]: Standard Cell Library: sky130_fd_sc_hd
[INFO]: Optimization Standard Cell Library: sky130_fd_sc_hd
[INFO]: Run Directory: /content/runs/RUN_2023.04.02_19.21.29
[INFO]: Preparing LEF files for the nom corner...
[INFO]: Preparing LEF files for the min corner...
[INFO]: Preparing LEF files for the max corner...
[STEP 1]
[INFO]: Running Synthesis (log: runs/RUN_2023.04.02_19.21.29/logs/synthesis/1-synthesis.log)...
^C


## Display layout

In [28]:
import gdstk
library = gdstk.read_gds('sky130_sram_1r1w_32x32_32.gds')
top_cells = library.top_level()
top_cells[0].write_svg('sky130_sram_1r1w_32x32_32.svg')
import cairosvg
cairosvg.svg2png(url='sky130_sram_1r1w_32x32_32.svg', write_to='sky130_sram_1r1w_32x32_32.png')
from IPython.display import Image
Image('sky130_sram_1r1w_64x16_32.png')

ModuleNotFoundError: ignored

## Metrics

[Documentation](https://openlane.readthedocs.io/en/latest/reference/datapoint_definitions.html)


In [ ]:
import pandas as pd
import pathlib

pd.options.display.max_rows = None
reports = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(reports[-1])
df.transpose()

,0
design,/content
design_name,inverter
config,RUN_2022.11.16_14.22.44
flow_status,flow completed
total_runtime,0h0m21s0ms
routed_runtime,0h0m11s0ms
(Cell/mm^2)/Core_Util,1014.898713
DIEAREA_mm^2,0.001971
CellPer_mm^2,507.449357
OpenDP_Util,-1
